In [1]:
import pandas as pd
import numpy as np
import dtale
from sklearn.model_selection import train_test_split
import seaborn as sn
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

Using TensorFlow backend.


In [11]:
# import from df from other notebook
%store -r df_scaled
%store -r df_full_info_scaled


In [12]:
df_scaled = df_full_info_scaled

In [13]:
# get X and y values
X = df_scaled.drop("TARGET", axis=1)
y = df_scaled["TARGET"]



In [14]:
# splitting while keeping the proportion of the TARGET
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify = y, random_state = 42)



balance_under = RandomUnderSampler('auto', random_state=42)
balance_df_under = balance_under.fit_resample(X_train,y_train)

balance_df = balance_df_under
X_train = balance_df[0]
y_train = np.array(balance_df[1])


In [33]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [34]:
y_pred = xgb_model.predict(X_test)


In [35]:
cm = confusion_matrix(y_test,y_pred)
cm

array([[4797, 1984],
       [ 212,  413]])

In [36]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y_pred)

0.6841088924937325

In [6]:
from numpy import mean
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold


In [15]:
#define model
model = XGBClassifier()

# define grid
weights = [0.5 , 1 , 1.5 , 2 , 3 , 4]
param_grid = dict(scale_pos_weight=weights)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X_train, y_train)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.740257 using {'scale_pos_weight': 0.5}
0.740257 (0.019408) with: {'scale_pos_weight': 0.5}
0.738247 (0.020215) with: {'scale_pos_weight': 1}
0.738624 (0.020219) with: {'scale_pos_weight': 1.5}
0.737959 (0.021471) with: {'scale_pos_weight': 2}
0.738513 (0.021839) with: {'scale_pos_weight': 3}
0.737774 (0.022118) with: {'scale_pos_weight': 4}


In [ ]:
from sklearn.metrics import roc_auc_score

y_prob = grid_result.predict_proba(X_test)[:,1]

roc_auc_score(y_test, y_pred)